<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/Q_and_A_model_deepset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install tika

In [46]:
import json
import os
import re
import sys
from tqdm import tqdm
from tika import parser
import pandas as pd
import uuid
import markdown
from pathlib import Path
from html.parser import HTMLParser

In [2]:
!pip install transformers==4.8.2

In [53]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'What is the value of a?',
    'context': "['a=10', 'b=20']"
}
res = nlp(QA_input)

In [54]:
res

{'answer': '10', 'end': 6, 'score': 0.7407091856002808, 'start': 4}

In [5]:
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q4-2019-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_news/archive/Q3-2019-Amazon-Financial-Results.pdf

--2021-07-01 15:06:31--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.2, 68.70.205.4, 68.70.205.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 945573 (923K) [application/pdf]
Saving to: ‘./docs/AMZN-Q3-2020-Earnings-Release.pdf’

AMZN-Q3-2020-Earnin 100%[===================>] 923.41K  --.-KB/s    in 0.06s   

2021-07-01 15:06:31 (14.1 MB/s) - ‘./docs/AMZN-Q3-2020-Earnings-Release.pdf’ saved [945573/945573]

--2021-07-01 15:06:31--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.2, 68.70.205.4, 68.70.205.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 366951 (358K) [application/pdf]
Saving to: ‘./docs/Q2-2020-Amazon-Ea

In [47]:
def pdf_converter(directory_path, min_length=200, include_line_breaks=False):

    list_file = os.listdir(directory_path)
    list_pdf = []
    for file in list_file:
        if file.endswith("pdf"):
            list_pdf.append(file)
    df = pd.DataFrame(columns=["title", "paragraphs"])
    for i, pdf in enumerate(list_pdf):
        try:
            df.loc[i] = [pdf.replace(".pdf",''), None]
            raw = parser.from_file(os.path.join(directory_path, pdf))
            s = raw["content"].strip()
            paragraphs = re.split("\n\n(?=\u2028|[A-Z-0-9])", s)
            list_par = []
            temp_para = "" 
            for p in paragraphs:
                if not p.isspace(): 
                    if include_line_breaks: 
                        if len(p) >= min_length:
                            if temp_para:
                                list_par.append(temp_para.strip())
                                temp_para = (
                                    ""
                                )  
                                list_par.append(
                                    p.replace("\n", "")
                                ) 
                            else:
                                list_par.append(p.replace("\n", ""))
                        else:
                            line = p.replace("\n", " ").strip()
                            temp_para = temp_para + f" {line}"
                    else:
                        list_par.append(p.replace("\n", ""))
                else:
                    if temp_para:
                        list_par.append(temp_para.strip())

            df.loc[i, "paragraphs"] = list_par
        except Exception as e:
            print("Unexpected error:", e)
            print("Unable to process file {}".format(pdf))
    return df

In [48]:
df = pdf_converter('/content/documents')

2021-07-01 15:39:16,496 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-07-01 15:39:17,270 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-07-01 15:39:17,768 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [57]:
print(str(df['paragraphs'][0]))

['Microsoft Word - Earnings Release_4.30.20_12PM with quote     ', 'AMAZON.COM ANNOUNCES FIRST QUARTER RESULTS ', 'SEATTLE—(BUSINESS WIRE) April 30, 2020—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2020.  •  Operating cash flow increased 16% to $39.7 billion for the trailing twelve months, compared with $34.4 billion for the trailing twelve months ended March 31, 2019. •  Free cash flow increased to $24.3 billion for the trailing twelve months, compared with $23.0 billion for the trailing twelve months ended March 31, 2019. •  Free cash flow less principal repayments of finance leases and financing obligations decreased to $14.3 billion for the trailing twelve months, compared with $15.1 billion for the trailing twelve months ended March 31, 2019. •  Free cash flow less equipment finance leases and principal repayments of all other finance leases and financing obligations decreased to $11.7 billion for the trailing twelve mont

In [69]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'What is the amount charged if Customer exceeds its total Image Serving limit?',
    'context': """Notwithstanding anything to the contrary, the License Term commences on the date Products and Services are made available to Customer or the License Term Start Date whichever is later. For a period up to 60 days prior to the License Term Start Date (the "Implementation Period"), but in no event earlier than the Effective Date, Customer may be granted a limited license to access the Products and Services for the sole purpose of implementation and testing on the Testing Instance. The Implementation Period is available for first time Campaign implementations, and not available for renewals or upsells. If Customer exceeds its total Image Serving limit, then Customer may be billed $1.00/GB per month in over-usage fees. If Customer exceeds its Total Database Storage limit, then Customer will be billed $30.00/GB per month in overusage fees. Adobe will invoice all overusage fees monthly in arrears, as incurred. Adobe Campaign Standard also includes Adobe Campaign Deliverability Fundamentals. During the IP warming phase, Deliverability Fundamental services may include: (1) Assessment of the existing marketing plan and creation of a customized ramp-up calendar based on Customer's existing practices, data, volume, and frequency of email; (2) Determination of the appropriate domains and number of IP addresses to be used; and (3) Recommendations for changes to the ramp-up calendar based on performance monitoring and reputation metrics. Limitations: Customer may consume up to forty-eight hours of Deliverability Fundamentals services, which will expire after the first twelve months of the License Term. Adobe will perform Services remotely unless travel reimbursement is otherwise stipulated in this Sales Order. Hours are not interchangeable with any other Adobe Professional Services project. Any hours that remain unused after the initial twelve months will expire and cannot be used for any other purpose. Adobe provides only one Deliverability Fundamentals package for new license purchases only, specifically excluding renewal or upsell. Adobe Campaign is licensed by Customer with the user interface language set to English (Universal). Customer cannot modify the user interface language."""
}
res = nlp(QA_input)

In [71]:
res['answer']

'$1.00/GB per month'